#Project 3
##Fundamentals of AI

In [1]:
import numpy as np
import random

##Question 2

In [2]:
def eval_state_action(V, s, a, gamma=0.9):

    return np.sum([rew+p * (gamma*V[(x,y)]) for p, (x,y), rew, _ in P[s][a]])
    '''
    Value iteration algorithm
    '''
def value_iteration(n=10):
    it = 0

    while it < n:
        delta = 0
        # update the value of each state and check for convergence
        for c in range(1,columns+1):
          for r in range(1,rows+1):
            old_v = V[(c,r)]
            if (c,r) not in [(1,5),(5,1),(5,3),(3,3)]:
              V[(c,r)] = np.max([eval_state_action(V, (c,r), a) for a in range(nA)]) #Get the Bellman updated value for all actions and choose the action which gives the maximum value
            #delta = max(delta, np.abs(old_v - V[c][r]))

        # if delta < eps:
        #     break
        # else:
        #print('Iter:', it, ' delta:', np.round(delta, 5))
        it += 1
    return V

def setProb(loc,pORd,p=0):
  if pORd in [4,5]:
    for a in range(nA):
            (c,r) = loc
            for d in range(nA):
                tup = P[(c,r)][a][d]
                temp = list(tup)
                temp[0] = 0.05
                P[(c,r)][a][d] = tuple(temp)
    tup= P[loc][pORd][pORd]
    temp=list(tup)
    temp[0]=p
    P[loc][pORd][pORd]=tuple(temp)
  else:
    for a in range(nA):
            (c,r) = loc
            for d in range(nA):
              if d in [0,1,2,3]:
                tup = P[(c,r)][a][d]
                temp = list(tup)
                temp[0] = 0.8 if a==d else 0.04
                P[(c,r)][a][d] = tuple(temp)
              else:
                tup = P[(c,r)][a][d]
                temp = list(tup)
                temp[0] = 0.04
                P[(c,r)][a][d] = tuple(temp)

def run_episodes(P, V, numb_epds = 1):
    '''
    Run some test games
    '''
    tot_rew = 0
    L = [(1,5),(5,1),(5,3)]
    for _ in range(numb_epds):
      P1,D1 = (1,5),(5,3) 
      P2,D2 = (5,1),(1,5) 
      state = T = (2,4)
      print('P1:',P1,'P2:',P2,'D1:',D1,'D2:',D2,'T:',T)

      V[P1] = 20000
      V[P2] = 20000
     
      V[(3,3)] = -100000
      setProb(P1,4,0.4)
      setProb(P2,4,0.4)
      path = []
      path.append(state)
      Picks = 2
      Drops = 2


      while Picks>0 or Drops>0:
        done = False
        V = value_iteration(10)

        while not done:
          values  = [eval_state_action(V, state, a) for a in range(nA)]
          actions = list(np.flatnonzero(values == np.max(values)))
          action = actions[0] 
          
          next_state = P[state][action][action][1]
          state = next_state
          if action not in [4,5]:
            path.append(state)

          if state in [(1,5),(5,1),(5,3)]:
            
            if action == 4:
              done = True
              c,r=state
              V[(c,r)] = 0 
              Picks-=1
              if P1 == P2:
                Picks-=1

              if state == P1:
                setProb(P1,0,0)
                setProb(D1,5,0.4)
                V[D1] = 20000
                  
              if state == P2:
                setProb(P2,0,0)
                setProb(D2,5,0.4)
                V[D2] = 20000

            if action == 5:
              done = True
              c,r=state
              V[(c,r)] = 0 #np.zeros((5,5))
              Drops-=1
              if D1 == D2:
                Drops-=1
              if Picks ==1:
                if D1 == P2 and state == D1:
                  Picks-=1
                  setProb(P2,0,0)
                  setProb(D2,5,0.4)
                  V[D2] = 20000
                elif D2 == P1 and state == D2:
                  Picks-=1
                  setProb(P1,0,0)
                  setProb(D1,5,0.4)
                  V[D1] = 20000
              if state==D1:
                setProb(D1,0,0)
              if state==D2:
                setProb(D2,0,0) 
             
      print('Path: ',path)
      print('Length of the path: ',len(path))
  
if __name__ == '__main__':
   
    # create the environment
    grid = np.ndarray((5,5),dtype=object)
    grid[2][2] = 'I'
    P = {}
    A = {0:'N',1:'S',2:'W',3:'E',4:'P',5:'D'}
    nA = len(A)
    #print(nA)
    (columns,rows) = grid.shape
    nS = columns*rows
    #print(nS)
    for c in range(1,columns+1):
      for r in range(1,rows+1):
        P[(c,r)] = {}
        if (c,r) in [(1,5),(5,1),(5,3)]:
          for a in range(nA):
            P[(c,r)][a] = []
            for d in range(nA):
              if d==0:
                P[(c,r)][a].append((0.05,(c,r+1 if r+1 in range(1,rows+1) else r),-1,False))
              elif d==1:
                P[(c,r)][a].append((0.05,(c,r-1 if r-1 in range(1,rows+1) else r),-1,False))
              elif d==2:
                P[(c,r)][a].append((0.05,(c-1 if c-1 in range(1,columns+1) else c,r),-1,False))
              elif d==3:
                P[(c,r)][a].append((0.05,(c+1 if c+1 in range(1,columns+1) else c,r),-1,False))
              else:
                P[(c,r)][a].append((0.05,(c,r),-1,False))
        else:
          for a in range(nA):
            P[(c,r)][a] = []
            for d in range(nA):
              if d==0:
                P[(c,r)][a].append((0.8 if a==d else 0.04,(c,r+1 if r+1 in range(1,rows+1) else r),-1,False))
              elif d==1:
                P[(c,r)][a].append((0.8 if a==d else 0.04,(c,r-1 if r-1 in range(1,rows+1) else r),-1,False))
              elif d==2:
                P[(c,r)][a].append((0.8 if a==d else 0.04,(c-1 if c-1 in range(1,columns+1) else c,r),-1,False))
              elif d==3:
                P[(c,r)][a].append((0.8 if a==d else 0.04,(c+1 if c+1 in range(1,columns+1) else c,r),-1,False))
              else:
                P[(c,r)][a].append((0.04,(c,r),-1,False))


    #print(P)
   
    V = {}
    for i in range(1,columns+1):
      for j in range(1,rows+1):
        V[(i,j)] = 0
  
    # test the value function on  episode 1
    run_episodes(P, V, numb_epds = 1)

P1: (1, 5) P2: (5, 1) D1: (5, 3) D2: (1, 5) T: (2, 4)
Path:  [(2, 4), (1, 4), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (5, 4), (5, 3), (5, 2), (5, 1), (4, 1), (3, 1), (2, 1), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
Length of the path:  19


##Question 3

In [3]:
def eval_state_action(V, s, a, gamma=0.9):      # function to calculate the state action

    return np.sum([rew+p * (gamma*V[(x,y)]) for p, (x,y), rew, _ in P[s][a]])
    '''
    Value iteration algorithm
    '''
def value_iteration(n=10):          #value iteration function
    it = 0

    while it < n:
        delta = 0
        # update the value of each state 
        for c in range(1,columns+1):
          for r in range(1,rows+1):
            old_v = V[(c,r)]
            if (c,r) not in [(1,5),(5,1),(5,3),(3,3)]:
              V[(c,r)] = np.max([eval_state_action(V, (c,r), a) for a in range(nA)]) #Get the Bellman updated value for all actions and choose the action which gives the maximum value
        it += 1
    return V

def setProb(loc,pORd,p=0):        # function to set the probability
  if pORd in [4,5]:               #to set probability for pick-up and dropoff locations.
    for a in range(nA):   
            (c,r) = loc
            for d in range(nA):
                tup = P[(c,r)][a][d]
                temp = list(tup)
                temp[0] = 0.05
                P[(c,r)][a][d] = tuple(temp)
    tup= P[loc][pORd][pORd]
    temp=list(tup)
    temp[0]=p
    P[loc][pORd][pORd]=tuple(temp)
  else:                                   # to set  probability for all other locations other than pick-up and dropoff locations
    for a in range(nA):
            (c,r) = loc
            for d in range(nA):
              if d in [0,1,2,3]:
                tup = P[(c,r)][a][d]
                temp = list(tup)
                temp[0] = 0.8 if a==d else 0.04
                P[(c,r)][a][d] = tuple(temp)
              else:
                tup = P[(c,r)][a][d]
                temp = list(tup)
                temp[0] = 0.04
                P[(c,r)][a][d] = tuple(temp)

def run_episodes(P, V, numb_epds = 5):      # function to run the episodes(5)
   
    tot_rew = 0
    L = [(1,5),(5,1),(5,3)]
    for _ in range(numb_epds):              #looping through 5 episodes  
      print('\nEpisode: ',_+1,'\n')
      P1,D1 = random.sample(L,k=2)      # setting pick-up and drop off locations at random
      P2,D2 = random.sample(L,k=2) 
      state = T = (2,4)                 # Initial taxi location
      print('P1:',P1,'P2:',P2,'D1:',D1,'D2:',D2,'T:',T)

      #setting rewards.
      V[P1] = 20000                    
      V[P2] = 20000
      V[(3,3)] = -100000

      setProb(P1,4,0.4)
      setProb(P2,4,0.4)
      path = []
      path.append(state)
      Picks = 2
      Drops = 2


      while Picks>0 or Drops>0:         # looping through sub-episodes
        done = False
        V = value_iteration(10)

        while not done:                 # steps through each sub-episodes.
          values  = [eval_state_action(V, state, a) for a in range(nA)]
          actions = list(np.flatnonzero(values == np.max(values)))
          action = actions[0] 
          
          next_state = P[state][action][action][1]
          state = next_state
          if action not in [4,5]:
            path.append(state)

          if state in [(1,5),(5,1),(5,3)]:
            
            if action == 4:                   # end sub-episode if pickup is done
              done = True
              c,r=state
              V[(c,r)] = 0 
              Picks-=1
              if P1 == P2:
                Picks-=1

              if state == P1:
                setProb(P1,0,0)
                setProb(D1,5,0.4)
                V[D1] = 20000
                  
              if state == P2:
                setProb(P2,0,0)
                setProb(D2,5,0.4)
                V[D2] = 20000

            if action == 5:                   # end sub-episode if drop off is done
              done = True
              c,r=state
              V[(c,r)] = 0 
              Drops-=1
              if D1 == D2:
                Drops-=1
              if Picks ==1:
                if D1 == P2 and state == D1:
                  Picks-=1
                  setProb(P2,0,0)
                  setProb(D2,5,0.4)
                  V[D2] = 20000
                elif D2 == P1 and state == D2:
                  Picks-=1
                  setProb(P1,0,0)
                  setProb(D1,5,0.4)
                  V[D1] = 20000
              if state==D1:
                setProb(D1,0,0)
              if state==D2:
                setProb(D2,0,0) 
             
      print('Path: ',path)
      print('Length of the path: ',len(path))
  
if __name__ == '__main__':                    # program execution starts from here
   
    # create the environment
    grid = np.ndarray((5,5),dtype=object)
    grid[2][2] = 'I'
    P = {}
    A = {0:'N',1:'S',2:'W',3:'E',4:'P',5:'D'}
    nA = len(A)
    (columns,rows) = grid.shape
    nS = columns*rows
    for c in range(1,columns+1):            
      for r in range(1,rows+1):
        P[(c,r)] = {}
        if (c,r) in [(1,5),(5,1),(5,3)]:
          for a in range(nA):
            P[(c,r)][a] = []
            for d in range(nA):
              if d==0:
                P[(c,r)][a].append((0.05,(c,r+1 if r+1 in range(1,rows+1) else r),-1,False))
              elif d==1:
                P[(c,r)][a].append((0.05,(c,r-1 if r-1 in range(1,rows+1) else r),-1,False))
              elif d==2:
                P[(c,r)][a].append((0.05,(c-1 if c-1 in range(1,columns+1) else c,r),-1,False))
              elif d==3:
                P[(c,r)][a].append((0.05,(c+1 if c+1 in range(1,columns+1) else c,r),-1,False))
              else:
                P[(c,r)][a].append((0.05,(c,r),-1,False))
        else:
          for a in range(nA):
            P[(c,r)][a] = []
            for d in range(nA):
              if d==0:
                P[(c,r)][a].append((0.8 if a==d else 0.04,(c,r+1 if r+1 in range(1,rows+1) else r),-1,False))
              elif d==1:
                P[(c,r)][a].append((0.8 if a==d else 0.04,(c,r-1 if r-1 in range(1,rows+1) else r),-1,False))
              elif d==2:
                P[(c,r)][a].append((0.8 if a==d else 0.04,(c-1 if c-1 in range(1,columns+1) else c,r),-1,False))
              elif d==3:
                P[(c,r)][a].append((0.8 if a==d else 0.04,(c+1 if c+1 in range(1,columns+1) else c,r),-1,False))
              else:
                P[(c,r)][a].append((0.04,(c,r),-1,False))


    # setting initial values of U to zero
    V = {}
    for i in range(1,columns+1):
      for j in range(1,rows+1):
        V[(i,j)] = 0
  
    # test the value function on 5 episodes
    run_episodes(P, V, numb_epds = 5)


Episode:  1 

P1: (5, 1) P2: (1, 5) D1: (1, 5) D2: (5, 1) T: (2, 4)
Path:  [(2, 4), (1, 4), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (4, 1), (3, 1), (2, 1), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
Length of the path:  19

Episode:  2 

P1: (5, 1) P2: (5, 1) D1: (1, 5) D2: (5, 3) T: (2, 4)
Path:  [(2, 4), (1, 4), (1, 3), (1, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (4, 5), (3, 5), (2, 5), (1, 5)]
Length of the path:  17

Episode:  3 

P1: (5, 1) P2: (5, 3) D1: (5, 3) D2: (1, 5) T: (2, 4)
Path:  [(2, 4), (2, 5), (3, 5), (4, 5), (5, 5), (5, 4), (5, 3), (5, 2), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (4, 5), (3, 5), (2, 5), (1, 5)]
Length of the path:  17

Episode:  4 

P1: (1, 5) P2: (5, 3) D1: (5, 3) D2: (5, 1) T: (2, 4)
Path:  [(2, 4), (2, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (5, 4), (5, 3), (5, 2), (5, 1)]
Length of the path:  11

Episode:  5 

P1: (1, 5) P2: (5, 1) D1: (5, 3) D2: (5, 3) T: (2, 4)
Path:  [(2, 4),